# Set-up

## Install required packages

In [ ]:
!pip install geopandas
!pip install Shapely

In [2]:
# CSV management
import pandas as pd

# Downloads and path management
import requests
import pathlib
import os

# Geographic analysis
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
from shapely.geometry import MultiPolygon
from shapely.geometry import MultiLineString
from shapely.ops import split

# Plotting
import matplotlib.pyplot as plt
from matplotlib import colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Numerical
import numpy as np

## Set filepaths

In [4]:
path = pathlib.Path().resolve()
INPUT_FILEPATH = path / "data"
OUTPUT_FILEPATH = path / "data" / "outputs"

# Prepare AFDC Data

## Download AFDC Data

In [4]:
api_key = "SEieCj3uYQ3ufDsCqFWAfw56GvXKyLr1YTt9K4Sv"
base_url = "https://developer.nrel.gov/api/alt-fuel-stations/v1"
download_format = "csv"
state = "CT"
fuel_type = "ELEC"

In [5]:
download_URL = base_url + "." + download_format + "?" + "api_key=" + api_key + "&" + "state=" + state + "&" + "fuel_type=" + fuel_type

In [9]:
# Download
data_df = pd.read_csv(download_URL)

In [10]:
# Save
data_df.to_csv(OUTPUT_FILEPATH /"afdc_ct_charging_points.csv")

In [30]:
# Load
data_df = pd.read_csv(OUTPUT_FILEPATH /"afdc_ct_charging_points.csv")

## Convert AFDC Data to GeoDataFrame

In [31]:
# geometry column must contain Point objects
geometry = [Point(x,y) for y,x in zip(data_df["Latitude"],data_df["Longitude"])]

In [33]:
# Create a new GeoDataFrame, set its geometry column
data_df_geo = data_df.copy()
data_df_geo["geometry"] = geometry

# Drop latitude and longitude, now superfluous columns
data_df_geo.drop(["Latitude","Longitude"], axis=1)

Unnamed: 0 Fuel Type Code  \
0             0           ELEC   
1             1           ELEC   
2             2           ELEC   
3             3           ELEC   
4             4           ELEC   
..          ...            ...   
797         797           ELEC   
798         798           ELEC   
799         799           ELEC   
800         800           ELEC   
801         801           ELEC   

                                          Station Name  \
0                                   Whole Foods Market   
1    Propark America - Omni Hotel Chapel Square Garage   
2    New London Parking Authority - Water Street Pu...   
3              Town of Westport - Westport Metro North   
4          Park 'N Fly - Bradley International Airport   
..                                                 ...   
797                          The Residence at Westport   
798                          Pierpont at City Crossing   
799                        STEPHEN TOYOTA FRONT CT4000   
800                                             Sunoco   
801                        Mystic Marriott Hotel & Spa   

                 Street Address  \
0              350 Grasmere Ave   
1                 155 Temple St   
2                  161 Water St   
3                 1 Railroad Pl   
4                 1186 South St   
..                          ...   
797         1141 Post Road East   
798                9 Tower Lane   
799         1097 Farmington Ave   
800                884 State St   
801  625 North Road (Route 117)   

                               Intersection Directions        City State  \
0                                                  NaN   Fairfield    CT   
1                     Located on first floor of garage   New Haven    CT   
2    Located outside off entrance from Water St and...  New London    CT   
3           Eastbound Westport Saugatuck Train Station    Westport    CT   
4                                                  NaN    Suffield    CT   
..                                                 ...         ...   ...   
797                                                NaN    Westport    CT   
798                                                NaN   New Haven    CT   
799                                                NaN     Bristol    CT   
800                                                NaN   New Haven    CT   
801                                                NaN      Groton    CT   

      ZIP  Plus4 Station Phone  ... Restricted Access RD Blends  \
0    6824    NaN  203-319-9544  ...              True       NaN   
1    6510    NaN  203-781-8030  ...             False       NaN   
2    6320    NaN  860-443-1775  ...             False       NaN   
3    6880    NaN  203-341-1000  ...             False       NaN   
4    6078    NaN  800-218-1297  ...             False       NaN   
..    ...    ...           ...  ...               ...       ...   
797  6880    NaN  888-998-2546  ...               NaN       NaN   
798  6519    NaN  888-998-2546  ...               NaN       NaN   
799  6010    NaN  888-758-4389  ...               NaN       NaN   
800  6511    NaN           NaN  ...             False       NaN   
801  6340    NaN  833-387-2643  ...               NaN       NaN   

    RD Blends (French) RD Blended with Biodiesel RD Maximum Biodiesel Level  \
0                  NaN                       NaN                        NaN   
1                  NaN                       NaN                        NaN   
2                  NaN                       NaN                        NaN   
3                  NaN                       NaN                        NaN   
4                  NaN                       NaN                        NaN   
..                 ...                       ...                        ...   
797                NaN                       NaN                        NaN   
798                NaN                       NaN                        NaN   
799                NaN                       NaN 

In [101]:
# Create the GeoDataFrame
data_df_geo = GeoDataFrame(data_df_geo,crs = "WGS84", geometry=geometry)

## Save and Load AFDC Data GeoDataFrame

In [34]:
# Save the geodataframe
data_df_geo.to_file(OUTPUT_FILEPATH / "afdc_ct_charging_points.gpkg")

# Load the dataframe
data_df_geo = gpd.read_file(OUTPUT_FILEPATH / "afdc_ct_charging_points.gpkg")

# Prepare Geographic Boundaries

## Prepare Connecticut State Boundaries

In [216]:
# Load all US States
us_gdf = gpd.read_file(INPUT_FILEPATH / "us_state_shapefiles")

In [217]:
# Restrict to CT
ct_gdf = us_gdf[us_gdf["NAME"] == "Connecticut"]

In [218]:
# Reset the index
ct_gdf = ct_gdf.reset_index()

In [219]:
ct_gdf = ct_gdf.drop('index', axis =1)

## Save CT state boundaries

In [249]:
ct_gdf.to_file(OUTPUT_FILEPATH / "ct_boundaries.gpkg")

## Prepare CT subcounty boundaries

In [235]:
# Import shapefile of CT counties
# Source: https://catalog.data.gov/dataset/tiger-line-shapefile-2019-state-connecticut-current-county-subdivision-state-based
counties_shapefile = gpd.read_file(INPUT_FILEPATH / "ct_counties_shapefile")

# Make it the same CRS
counties_shapefile = counties_shapefile.to_crs(ct_gdf.crs)

In [236]:
counties_shapefile.head()

STATEFP COUNTYFP COUSUBFP  COUSUBNS       GEOID  \
0      09      005    60750  00213489  0900560750   
1      09      001    00000  00000000  0900100000   
2      09      001    56060  00213481  0900156060   
3      09      001    04720  00213390  0900104720   
4      09      001    08070  00213396  0900108070   

                              NAME                         NAMELSAD LSAD  \
0                         Plymouth                    Plymouth town   43   
1  County subdivisions not defined  County subdivisions not defined   00   
2                          Norwalk                     Norwalk town   43   
3                           Bethel                      Bethel town   43   
4                       Bridgeport                  Bridgeport town   43   

  CLASSFP  MTFCC CNECTAFP NECTAFP  NCTADVFP FUNCSTAT     ALAND     AWATER  \
0      T1  G4040      790   73450       NaN        A  56650624    1199855   
1      Z9  G4040      NaN     NaN       NaN        F         0  302385073   
2      T5  G4040      720   71950       NaN        C  59275235   34928558   
3      T1  G4040      720   72850       NaN        A  43918201     156161   
4      T5  G4040      720   71950       NaN        C  41606698    8705768   

      INTPTLAT      INTPTLON  \
0  +41.6663915  -073.0265164   
1  +41.0779471  -073.2165184   
2  +41.0927388  -073.4197955   
3  +41.3697777  -073.3895032   
4  +41.1873933  -073.1957567   

                                            geometry  
0  POLYGON ((-73.06467 41.64182, -73.06439 41.643...  
1  POLYGON ((-73.45646 40.99116, -73.45646 41.013...  
2  POLYGON ((-73.47456 41.11470, -73.47441 41.114...  
3  POLYGON ((-73.43502 41.37240, -73.43457 41.373...  
4  POLYGON ((-73.24409 41.22660, -73.24340 41.226...

## Save subcounty boundaries

In [248]:
counties_shapefile.to_file(OUTPUT_FILEPATH / "ct_subcounties.gpkg")

In [9]:
counties_shapefile = gpd.read_file(OUTPUT_FILEPATH / "ct_subcounties.gpkg")

## Prepare CT ZIP boundaries

Second, import ZIP shapefile

In [226]:
us_zips = gpd.read_file(INPUT_FILEPATH / "us_zips")

In [231]:
ct_zips = us_zips[us_zips["ZCTA5CE20"].astype(str).str[0:2]=="06"]

In [237]:
ct_zips = ct_zips.to_crs(ct_gdf.crs)

## Save CT ZIP boundaries

In [246]:
ct_zips.to_file(OUTPUT_FILEPATH / "ct_zips.gpkg")

# Prepare Routes

## Import route data

In [21]:
# Add state routes
# Source: https://geodata.ct.gov/maps/5e334f57a37f4eaa970b0ee5a52624d0/about
state_routes = gpd.read_file(INPUT_FILEPATH / "CTDOT_State_Routes_and_Local_Roads.geojson")

# Source: https://geodata.ct.gov/maps/5e334f57a37f4eaa970b0ee5a52624d0/about
local_roads = gpd.read_file(INPUT_FILEPATH / "CTDOT_State_Routes_and_Local_Roads_LOCAL")

# Source: https://geodata.ct.gov/datasets/b878865b7c4e4d858fbddf92ed4e1c28_0/explore
interstates = gpd.read_file(INPUT_FILEPATH / "Connecticut_Routes.geojson")

# Remove state routes for duplication
interstates = interstates[interstates["ROUTECLASS"]!="State Route"]

# Ramps
# Source: https://connecticut-ctdot.opendata.arcgis.com/apps/76c04dd1fb5b4933a0aefa12c632457c/explore
ramps = gpd.read_file(INPUT_FILEPATH / "ramps.geojson")

## Reproject route data

In [38]:
# Reproject to use the same coordinate system
state_routes= state_routes.to_crs(ct_gdf.crs)
local_roads = local_roads.to_crs(ct_gdf.crs)
interstates = interstates.to_crs(ct_gdf.crs)
ramps = ramps.to_crs(ct_gdf.crs)

## Save the route data

In [247]:
state_routes.to_file(OUTPUT_FILEPATH / "state_routes.gpkg")
local_roads.to_file(OUTPUT_FILEPATH / "local_roads.gpkg")
interstates.to_file(OUTPUT_FILEPATH / "interstates.gpkg")
ramps.to_file(OUTPUT_FILEPATH / "ramps.gpkg")

# Merge Demographic Data

In [11]:
# Now join the census data to the counties
census_data_s1901 = pd.read_csv(INPUT_FILEPATH/ "S1901_CT.csv")

# Convert the shapefile countyfp to numeric
counties_shapefile["COUNTYFP"] = pd.to_numeric(counties_shapefile["COUNTYFP"])

# Join the data
ct_counties_joined = counties_shapefile.merge(census_data_s1901, left_on = "COUNTYFP", right_on = "county")

## Save merged demographic data

In [14]:
ct_counties_joined.to_file(OUTPUT_FILEPATH / "ct_subcounties_income_merged.gpkg")